In [ ]:
!pip install caer canaro

In [ ]:
import os
import caer
import canaro
import numpy as np
import cv2
import gc
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
IMG_SIZE = (80,80)
channels = 1
char_path = r'../input/the-simpsons-characters-dataset/simpsons_dataset'

In [ ]:
char_dict = {}
for char in os.listdir(char_path):
    char_dict[char] = len(os.listdir(os.path.join(char_path, char)))

# Sort in descending order
char_dict = caer.sort_dict(char_dict, descending=True)
char_dict

In [ ]:
characters = []
count = 0
for i in char_dict:
    characters.append(i[0])
    count += 1
    if count >= 10:
        break

In [ ]:
train = caer.preprocess_from_dir(char_path, characters, channels=channels, IMG_SIZE=IMG_SIZE, isShuffle=True)

In [ ]:
len(train)

In [ ]:
plt.figure(figsize=(30,30))
plt.imshow(train[0][0], cmap='gray')
plt.show()

In [ ]:
featureSet, labels = caer.sep_train(train, IMG_SIZE=IMG_SIZE)

In [ ]:
from tensorflow.keras.utils import to_categorical
# Normalize featureSet
featureSet = caer.normalize(featureSet)
labels = to_categorical(labels, len(characters))

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(featureSet, labels, test_size=0.2)

# split_data = skm.train_test_split(featureSet, labels, test_size=.2)
# x_train, x_val, y_train, y_val = (np.array(item) for item in split_data)

In [ ]:
del train
del featureSet
del labels
gc.collect()

In [ ]:
BATCH_SIZE = 32
EPOCH = 10

In [ ]:
# Image data generator
datagen = canaro.generators.imageDataGenerator()
train_gen = datagen.flow(x_train, y_train, batch_size=BATCH_SIZE)

In [ ]:
model = canaro.models.createSimpsonsModel(IMG_SIZE = IMG_SIZE, channels=channels, 
                                          output_dim=len(characters), loss='binary_crossentropy',
                                         decay=1e-6, learning_rate=0.001, momentum=0.9, nesterov=True)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]

In [ ]:
training = model.fit(train_gen,
                     steps_per_epoch=len(x_train)//BATCH_SIZE,
                    epochs=EPOCH,
                    validation_data=(x_val, y_val),
                    validation_steps=len(y_val)//BATCH_SIZE,
                    callbacks=callbacks_list)

In [ ]:
"""## Testing"""

test_path = r'../input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/charles_montgomery_burns_0.jpg'

img = cv.imread(test_path)

plt.imshow(img)
plt.show()

In [ ]:
def prepare(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image = cv.resize(image, IMG_SIZE)
    image = caer.reshape(image, IMG_SIZE, 1)
    return image

In [ ]:
predictions = model.predict(prepare(img))

# Getting class with the highest probability
print(characters[np.argmax(predictions[0])])